In [24]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
from IPython.display import display
import pandas as pd
import nltk
from IPython.display import display
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import tkinter as tk
from tkinter import ttk
import pandas as pd

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\basan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\basan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\basan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\basan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
import requests
from bs4 import BeautifulSoup


def get_url(url):
    try:
        # Send a GET request to the specified URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, "html.parser")
            # Find the anchor tag with class "portal_link" and text "Research output"
            research_output_link_element = soup.find("a",
                                                     class_="portal_link",
                                                     text="Research output")

            if research_output_link_element:
                # Get the URL linked to the element
                research_output_link = research_output_link_element["href"]

                # Construct the full URL if it's relative
                if research_output_link.startswith("/"):
                    research_output_link = "https://pureportal.coventry.ac.uk" + research_output_link

                # Return the research output URL
                return research_output_link
            else:
                print("Error: Research output element not found on the page.")
                return None
        else:
            # If the request was not successful, print an error message
            print(
                "Error: Unable to fetch data from the specified URL. Status code:",
                response.status_code)
            return None
    except Exception as e:
        # Handle any exceptions that occur during the process
        print("An error occurred:", str(e))
        return None

In [29]:
# Define the URL
url = "https://pureportal.coventry.ac.uk/en/organisations/ihw-centre-for-health-and-life-sciences-chls/"

# Call the function and save the result into the url variable
url = get_url(url)

C:\Users\basan\AppData\Local\Temp\ipykernel_20320\3441163301.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  research_output_link_element = soup.find("a",


In [13]:
response = requests.get(url)
response

<Response [200]>

In [31]:
def create_csv():
    with open("publication.csv", "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Title", "Authors", "Publication Year", "Publication Link", "Authors Profile"])
    print("CSV file created successfully.")

    
def update_csv(database):
    current_data = pd.read_csv(database, index_col="Unnamed: 0")
    return current_data       


In [32]:
sample_db = pd.read_csv('publication.csv').reset_index().rename(columns={'index':'SN'})

# print(f'{sample_db.shape[0]} records were scraped')

In [33]:
# Remove stop words
import string
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


def tp1(txt):
    txt = txt.lower()   # Make lowercase
    txt = txt.translate(str.maketrans('','',string.punctuation))   # Remove punctuation marks
    txt = lematize(txt)
    return txt


def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax


In [17]:
processed_db = sample_db.copy()

def preprocess_df(df):
    df.Title = df.Title.apply(tp1)
    df.Author = df.Authors.str.lower()
    df = df.drop(columns=['Authors','Publication Year'], axis=1)
    return df
    
preprocess_df(processed_db)
processed_db.head()

C:\Users\basan\AppData\Local\Temp\ipykernel_20320\2613971725.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.Author = df.Authors.str.lower()


,SN,Title,Authors,Publication Year,Publication Link,Authors Profile
0,0,barrier facilitator participation key componen...,"Roden, L.",1 Jan 2024,https://pureportal.coventry.ac.uk/en/publicati...,https://pureportal.coventry.ac.uk/en/persons/l...
1,1,doxorubicin alters gprotein couple receptormed...,"Lozahic, C.,Maddock, H.,Wheatley, M.,Sandhu, H.",29 Jan 2024,https://pureportal.coventry.ac.uk/en/publicati...,https://pureportal.coventry.ac.uk/en/persons/c...
2,2,human gut microbiota endocrinology paradigm sh...,"Turner, M. C.,Morozov, I.",1 Feb 2024,https://pureportal.coventry.ac.uk/en/publicati...,https://pureportal.coventry.ac.uk/en/persons/m...
3,3,hatem ali,Hatem Ali,1 Feb 2024,https://pureportal.coventry.ac.uk/en/persons/h...,https://pureportal.coventry.ac.uk/en/persons/h...
4,4,sara anisi,Sara Anisi,1 Feb 2024,https://pureportal.coventry.ac.uk/en/persons/s...,https://pureportal.coventry.ac.uk/en/persons/s...


In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd

class InvertedIndex:
    def __init__(self):
        self.index = {}

    def index_document(self, doc_id, text, field):
        # Tokenize text into terms
        terms = text.split()
        # Update index with terms and corresponding document IDs
        for term in terms:
            if term not in self.index:
                self.index[term] = {}
            if doc_id not in self.index[term]:
                self.index[term][doc_id] = set()
            self.index[term][doc_id].add(field)

    def search(self, query, field):
        # Tokenize query into terms
        terms = query.split()
        # Initialize result set with documents containing the first term
        results = set(self.index.get(terms[0], {}).keys())
        # Iterate through remaining terms and refine results
        for term in terms[1:]:
            results = results.intersection(self.index.get(term, {}).keys())
        # Filter results by selected field
        return [doc_id for doc_id in results if field.lower() in df.loc[doc_id].to_string().lower()]
df = processed_db
data = {
    "SN": df['SN'].tolist(),
    "Title": df['Title'].tolist(),
    "Authors": df['Authors'].tolist(),
    "Publication Year":df['Publication Year'].tolist(),
    "Publication Link":df['Publication Link'].tolist(),
    "Authors Profile":df['Authors Profile'].tolist()
    
    
}
# Initialize an inverted index
index = InvertedIndex()

# Indexing publications dynamically based on field
def index_based_on_field(field):
    index.index = {}
    for doc_id, row in df.iterrows():
        publication_data = row[field]
        index.index_document(doc_id, publication_data, field)

# Example usage
field_to_index = "Title"  # Change this to "Authors" to index based on authors
index_based_on_field(field_to_index)

# Create a Tkinter application
root = tk.Tk()
root.title("Inverted Index Search")

# Create a frame to hold the search input and results
frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Create a label for the search query
ttk.Label(frame, text="Search Query:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
query_entry = ttk.Entry(frame, width=50)
query_entry.grid(row=0, column=1, padx=5, pady=5, sticky=(tk.W, tk.E))

# Create a dropdown menu for selecting the search field
selected_field = tk.StringVar()
selected_field.set("Title")
def update_selected_field(value):
    global field_to_index
    selected_field.set(value)
    field_to_index = value
    index_based_on_field(field_to_index)  # Re-index based on the selected field
field_options = ["Title", "Authors"]
field_dropdown = ttk.OptionMenu(frame, selected_field, *field_options, command=update_selected_field)
field_dropdown.grid(row=0, column=2, padx=5, pady=5, sticky=tk.E)

# Create a treeview to display search results in table format
result_columns = list(df.columns)
result_tree = ttk.Treeview(frame, columns=result_columns, show="headings")
for col in result_columns:
    result_tree.heading(col, text=col)
result_tree.grid(row=1, column=0, columnspan=3, padx=5, pady=5, sticky=(tk.W, tk.E))

# Function to handle search button click event
def search():
    query = query_entry.get()
    field = selected_field.get()
    results = index.search(query, field)
    result_tree.delete(*result_tree.get_children())
    for doc_id in results:
        row_data = df.loc[[doc_id]].values.tolist()[0]
        result_tree.insert("", "end", values=row_data)

# Create a search button
search_button = ttk.Button(frame, text="Search", command=search)
search_button.grid(row=0, column=3, padx=5, pady=5, sticky=tk.E)

root.mainloop()


In [27]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import pandas as pd

class InvertedIndex:
    def __init__(self):
        self.index = {}

    def index_document(self, doc_id, text, field):
        # Tokenize text into terms
        terms = text.split()
        # Update index with terms and corresponding document IDs
        for term in terms:
            if term not in self.index:
                self.index[term] = {}
            if doc_id not in self.index[term]:
                self.index[term][doc_id] = set()
            self.index[term][doc_id].add(field)

    def search(self, query, field):
        # Tokenize query into terms
        terms = query.split()
        # Initialize result set with documents containing the first term
        results = set(self.index.get(terms[0], {}).keys())
        # Iterate through remaining terms and refine results
        for term in terms[1:]:
            results = results.intersection(self.index.get(term, {}).keys())
        # Filter results by selected field
        return [doc_id for doc_id in results if field.lower() in df.loc[doc_id].to_string().lower()]

df = sample_db

# Initialize an inverted index
index = InvertedIndex()

# Indexing publications dynamically based on field
def index_based_on_field(field):
    index.index = {}
    for doc_id, row in df.iterrows():
        publication_data = row[field]
        index.index_document(doc_id, publication_data, field)

# Example usage
field_to_index = "Title"  # Change this to "Authors" to index based on authors
index_based_on_field(field_to_index)

# Create a Tkinter application
root = tk.Tk()
root.title("Inverted Index Search")

# Load and resize the logo
logo_image = Image.open("coventry-university-logo.png")
logo_image = logo_image.resize((100, 100), Image.ANTIALIAS)
logo_photo = ImageTk.PhotoImage(logo_image)

# Create a frame to hold the search input and results
frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Display the logo in the top left corner
logo_label = ttk.Label(frame, image=logo_photo)
logo_label.image = logo_photo  # Keep a reference
logo_label.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)

# Search Engine Title Label
search_engine_label = ttk.Label(frame, text="Search engine", font=("Arial Bold", 18))
search_engine_label.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)

# Create a label for the search query
ttk.Label(frame, text="Search Query:").grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
query_entry = ttk.Entry(frame, width=50)
query_entry.grid(row=1, column=1, padx=5, pady=5, sticky=(tk.W, tk.E))

# Create a dropdown menu for selecting the search field
selected_field = tk.StringVar()
selected_field.set("Title")
def update_selected_field(value):
    global field_to_index
    selected_field.set(value)
    field_to_index = value
    index_based_on_field(field_to_index)  # Re-index based on the selected field
field_options = ["Title", "Authors"]
field_dropdown = ttk.OptionMenu(frame, selected_field, *field_options, command=update_selected_field)
field_dropdown.grid(row=1, column=2, padx=5, pady=5, sticky=tk.E)

# Create a treeview to display search results in table format
result_columns = list(df.columns)
result_tree = ttk.Treeview(frame, columns=result_columns, show="headings")
for col in result_columns:
    result_tree.heading(col, text=col)
result_tree.grid(row=2, column=0, columnspan=3, padx=5, pady=5, sticky=(tk.W, tk.E))

# Function to handle search button click event
def search():
    query = query_entry.get()
    field = selected_field.get()
    results = index.search(query, field)
    result_tree.delete(*result_tree.get_children())
    for doc_id in results:
        row_data = df.loc[[doc_id]].values.tolist()[0]
        result_tree.insert("", "end", values=row_data)

# Create a search button
search_button = ttk.Button(frame, text="Search", command=search)
search_button.grid(row=1, column=3, padx=5, pady=5, sticky=tk.E)

root.mainloop()


AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'